In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

import models
import data_generation
import evaluation

import os

# import ray
# ray.init(num_cpus=30)


In [2]:
beta_1=np.append(np.array([4,4,-8,-4,6,-4,8,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,2,-4,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,3,-6,-3,4.5,6]),np.zeros(90))

beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

In [3]:
n1=200
n2=200
n3=200
p=100
corval=0
prior=[0,1,2,3,4]

In [4]:
vs = np.zeros((1,100))

In [5]:
for i in range(1):
    data_generation.setup_seed(i)
    x1,y1=data_generation.generate_data_linear(corval,beta_1,n1,p)
    x2,y2=data_generation.generate_data_linear(corval,beta_2,n2,p)
    x3,y3=data_generation.generate_data_linear(corval,beta_3,n3,p)
    
    train_x1, test_x1, train_y1, test_y1= train_test_split(x1,y1, test_size=0.2, random_state=1)
    train_x2, test_x2, train_y2, test_y2= train_test_split(x2,y2, test_size=0.2, random_state=1)
    train_x3, test_x3, train_y3, test_y3= train_test_split(x3,y3, test_size=0.2, random_state=1)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
################## first step######################################  
    prior=[0,1,2,3,4]
# choose lambda1,lambda2 by ray tune
    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.001,0.002,0.003]),
            "ga":tune.choice([0.3,0.4,0.5]),
            "inputs_1":train_x1,
            "inputs_2":train_x2,
            "inputs_3":train_x3,
           "targets_1":train_y1,
           "targets_2":train_y2,
           "targets_3":train_y3,
            "prior":prior
        }

    result = tune.run(
            models.train_model_main_prior,
            config=config,
            num_samples=1)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    y_prior_1,y_prior_2,y_prior_3=models.main_prior_model(final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["lr"],final_lambda["ga"],
                           inputs_1,inputs_2,inputs_3,targets_1,targets_2,targets_3,prior) 
    
################## third step##################
    no_prior=[]

    config={
                "lambda1": tune.choice([0.03,0.04,0.05]),
                "lambda2": tune.choice([0.001,0.002,0.003]),
                 "lr": tune.choice([0.7,0.8,0.9]),
                "eta": tune.choice([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
                 "ga":tune.choice([0.3]),
                "inputs_1":train_x1,
                "inputs_2":train_x2,
                "inputs_3":train_x3,
                "train_y1":train_y1,
                "train_y2":train_y2,
                "train_y3":train_y3,
                "y_prior_1":y_prior_1,
                "y_prior_2":y_prior_2,
                "y_prior_3":y_prior_3,
                "prior":no_prior
            }

    result = tune.run(
            models.train_model_main_final,  
            config=config,
            num_samples=1)

################## fourth step##################

    #no prior
    no_prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    prediction,target,model_1_weight,model_2_weight,model_3_weight=models.main_final_model(final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["lr"],final_parameter["ga"],final_parameter["eta"],
                       inputs_1,inputs_2,inputs_3,train_y1,train_y2,train_y3,y_prior_1,y_prior_2,y_prior_3,
                       test_inputs_1,test_inputs_2,test_inputs_3,test_targets_1,test_targets_2,test_targets_3,no_prior)
    
####################################   prediction ####################################      

    final_matrix=confusion_matrix(target,prediction)
    TPR=recall_score(target,prediction)#TPR
    TNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
    GM = math.sqrt(TPR*TNR)
    accuracy=accuracy_score(target,prediction)
    precision=precision_score(target,prediction)
    f1=f1_score(target,prediction)
    
    
####################################   variable selection ####################################     
    index,TP,FP,FN,TN=evaluation.variable_selection(0.12,model_1_weight,model_2_weight,model_3_weight,beta_1,p)

    vsSEN=TP/(TP+FN)
    vsSPE=(TN/(TN+FP))
    vsGM=math.sqrt(vsSEN*vsSPE)
    vsMR=(FP+FN)/(TP+FN+TN+FP)
    vsCCR=1-vsMR
    
    vs[:,list(index)]+=1
    
    print(i)
    
    log_file_name = "{}.csv".format("PPIN")
    log_file_dir = "./results/simulation/"
    log_file_path = os.path.join(log_file_dir, log_file_name)

    with open(log_file_path, "a") as f:
        logs= "{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(corval, len(prior),vsGM,vsSEN,vsSPE,vsMR,vsCCR,GM,TPR,TNR,accuracy,precision,f1)
        f.writelines(logs)
    

2022-11-30 14:38:05,519	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2022-11-30 14:38:06,499	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,ga,lambda1,lambda2,lr,iter,total time (s),my_test_BCE_loss
train_model_main_prior_8c450_00000,TERMINATED,10.224.0.12:11286,0.3,0.03,0.003,0.8,1,7.5794,1.34117


(raylet) [2022-11-30 14:38:15,403 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 265498624; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_main_prior_8c450_00000:
  date: 2022-11-30_14-38-16
  done: false
  experiment_id: 3b3a54cfd850477298bfa1026540ccf5
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.3411664724349976
  node_ip: 10.224.0.12
  pid: 11286
  time_since_restore: 7.579400539398193
  time_this_iter_s: 7.579400539398193
  time_total_s: 7.579400539398193
  timestamp: 1669790296
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 8c450_00000
  warmup_time: 0.006536722183227539
  
Result for train_model_main_prior_8c450_00000:
  date: 2022-11-30_14-38-16
  done: true
  experiment_id: 3b3a54cfd850477298bfa1026540ccf5
  experiment_tag: 0_ga=0.3000,lambda1=0.0300,lambda2=0.0030,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.3411664724349976
  node_ip: 10.224.0.12
  pid: 11286
  time_since_restore: 7.579400539398193
  time_this_iter_s: 7.579400539398193
  time_total_s: 7.579400539398193
  timestamp: 16

2022-11-30 14:38:16,602	INFO tune.py:758 -- Total run time: 10.11 seconds (9.94 seconds for the tuning loop).


...

2022-11-30 14:38:29,391	INFO tune.py:758 -- Total run time: 11.31 seconds (11.17 seconds for the tuning loop).


0


(raylet) [2022-11-30 14:38:35,420 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263782400; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:38:45,428 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263782400; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:38:55,438 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263774208; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:39:05,456 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263741440; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 14:43:45,721 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263274496; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:43:55,730 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263274496; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:44:05,740 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263249920; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:44:15,750 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263241728; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 14:48:56,009 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 259846144; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:49:06,018 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 259813376; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:49:16,028 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 259723264; capacity: 29394726912. Object creation will fail if spilling is required.


In [31]:
vs[:,list(index)]+=1

(raylet) [2022-11-30 14:36:46,261 E 28499 28524] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_13-56-31_505409_28385 is over 95% full, available space: 271077376; capacity: 29394726912. Object creation will fail if spilling is required.


In [32]:
vs

array([[1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])